In [ ]:
# Classificação da Cobertura do Solo Através do Método LightGBM.
# Universidade Federal de Santa Catarina
# Natacha Pires Ramos


# PARA RODAR ESSE SCRIPT, É NECESSÁRIO O DOWNLOAD DAS IMAGENS DO SATÉLITE SENTINEL 2 DA ÁREA QUE DESEJA SER ESTUDADA, JUNTAMENTE COM SUAS RESPECTIVAS BANDAS E RESOLUÇÕES.
# ISSO PODE SER FEITO PELO SITE DO COPERNICUS BROWSER (https://browser.dataspace.copernicus.eu)

In [ ]:
# IMPORTANDO AS BIBLIOTECAS NECESSÁRIAS

import os
import rasterio
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


In [ ]:
# CONFIGURANDO OS DADOS DE ENTRADA
# Exemplo: band_dir_data2 = r"C:\Caminho\Para\SuaImagem\R10m"


# Caminho para o diretório da primeira imagem (data mais antiga)
band_dir_data1 = r" Coloque o doretório aqui "

# Caminho para o diretório da segunda imagem (data mais recente)

band_dir_data2 = r" Coloque o diretório aqui "


In [ ]:
# DEFINIÇÃO DAS CLASSES PARA A CLASSIFICAÇÃO DO SOLO

class_labels = {
    1: 'Vegetação',
    2: 'Água',
    3: 'Urbano',
    4: 'Solo Exposto',
    5: 'Agricultura'
}
class_colors = ['green', 'blue', 'gray', 'saddlebrown', 'yellow']

In [ ]:
# DEFININDO AS FUNÇÕES AUXILIARES PARA CALCULAR NDVI, EVI, NDWI
#Encontra o arquivo de uma banda específica no diretório.
#Calcula os índices NDVI, EVI e NDWI a partir dos caminhos das bandas.


def find_band_file(band_dir, band_prefix):   
    for file in os.listdir(band_dir):
        if band_prefix in file and file.endswith('.jp2'):
            return os.path.join(band_dir, file)
    raise FileNotFoundError(f"Arquivo da banda {band_prefix} não encontrado em {band_dir}")

def calculate_indices(band_paths):
    with rasterio.open(band_paths['B2']) as b2:
        blue = b2.read(1).astype('float32') / 10000
        profile = b2.profile
    with rasterio.open(band_paths['B3']) as b3:
        green = b3.read(1).astype('float32') / 10000
    with rasterio.open(band_paths['B4']) as b4:
        red = b4.read(1).astype('float32') / 10000
    with rasterio.open(band_paths['B8']) as b8:
        nir = b8.read(1).astype('float32') / 10000

    np.seterr(divide='ignore', invalid='ignore')
    
    ndvi = (nir - red) / (nir + red)
    evi = 2.5 * (nir - red) / (nir + 6 * red - 7.5 * blue + 1)
    evi = np.clip(evi, -1, 1)
    ndwi = (green - nir) / (green + nir)
    
    return ndvi, evi, ndwi, profile


# PARA VISUALIZAR AS FIGURAS DE NDVI, EVI E NDWI CALCULADAS PARA A REGIÃO DE ESTUDO, DESCOMENTE A SEÇÃO
# fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))

# im1 = ax1.imshow(ndvi, cmap='RdYlGn', vmin=-1, vmax=1)
# ax1.set_title('NDVI')
# ax1.set_axis_off() 
# fig.colorbar(im1, ax=ax1, orientation='horizontal', fraction=0.05, pad=0.04)

# im2 = ax2.imshow(evi, cmap='viridis', vmin=-1, vmax=1)
# ax2.set_title('EVI')
# ax2.set_axis_off()
# fig.colorbar(im2, ax=ax2, orientation='horizontal', fraction=0.05, pad=0.04)

# im3 = ax3.imshow(ndwi, cmap='RdYlBu', vmin=-1, vmax=1)
# ax3.set_title('NDWI')
# ax3.set_axis_off()
# fig.colorbar(im3, ax=ax3, orientation='horizontal', fraction=0.05, pad=0.04)

# plt.tight_layout()
# plt.show()

In [ ]:
# AGORA VAMOS DEFINIR OS PARÂMETROS QUE SERÃO USADOS NO MODELO 


def generate_samples_melhorado(n_per_class=200):
    """
    Gera dados sintéticos com faixas de valores mais realistas e amplas
    para treinamento do modelo de classificação de cobertura do solo.
    """
    np.random.seed(42)
    samples = []

    def sample_range(low, high, size):
        return np.random.uniform(low, high, size)
    
    # Vegetação
    for _ in range(n_per_class):
        samples.append({
            'NDVI': sample_range(0.2, 1.0, 1)[0],
            'EVI': sample_range(0.3, 1.0, 1)[0],
            'NDWI': sample_range(-0.2, 0.3, 1)[0],
            'label': 1
        })

    # Água
    for _ in range(n_per_class):
        samples.append({
            'NDVI': sample_range(-1.0, 0.0, 1)[0],
            'EVI': sample_range(-1.0, 0.0, 1)[0],
            'NDWI': sample_range(0.2, 1.0, 1)[0],
            'label': 2
        })

    # Urbano
    for _ in range(n_per_class):
        samples.append({
            'NDVI': sample_range(0.0, 0.3, 1)[0],
            'EVI': sample_range(0.0, 0.2, 1)[0],
            'NDWI': sample_range(-0.3, 0.2, 1)[0],
            'label': 3
        })

    # Solo exposto
    for _ in range(n_per_class):
        samples.append({
            'NDVI': sample_range(0.1, 0.4, 1)[0],
            'EVI': sample_range(0.1, 0.3, 1)[0],
            'NDWI': sample_range(-0.3, 0.1, 1)[0],
            'label': 4
        })

    # Agricultura
    for _ in range(n_per_class):
        samples.append({
            'NDVI': sample_range(0.3, 0.6, 1)[0],
            'EVI': sample_range(0.2, 0.5, 1)[0],
            'NDWI': sample_range(-0.2, 0.3, 1)[0],
            'label': 5
        })



    return pd.DataFrame(samples)



#Função completa para processar uma imagem e retornar o mapa classificado.
def processar_e_classificar_imagem(band_dir, model):
    print(f"\nProcessando imagem do diretório: {os.path.basename(os.path.dirname(os.path.dirname(band_dir)))}")
    
    # Mapeia as bandas dinamicamente
    band_paths = {
        'B2': find_band_file(band_dir, '_B02_10m'),
        'B3': find_band_file(band_dir, '_B03_10m'),
        'B4': find_band_file(band_dir, '_B04_10m'),
        'B8': find_band_file(band_dir, '_B08_10m')
    }
    
    # Calcula os índices
    ndvi, evi, ndwi, profile = calculate_indices(band_paths)
    
    # Aplica máscara de água antes da classificação
    water_mask = ndwi > 0.3
    
    # Prepara os dados para o modelo
    X_image = pd.DataFrame({
        'NDVI': ndvi.flatten(),
        'EVI': evi.flatten(),
        'NDWI': ndwi.flatten()
    })
    X_image.fillna(0, inplace=True) # Preenche NaNs para evitar erros na predição
    
    # Predição com o modelo treinado
    predicted_labels = model.predict(X_image)
    
    # Remodelando para o formato da imagem
    classified_image = predicted_labels.reshape(ndvi.shape)
    
    # Na imagem final classificada, forçamos a máscara de água para garantir a classe 'Água'
    classified_image[water_mask] = 2 # 2 é o rótulo para 'Água'
    
    return classified_image, profile


In [ ]:
# TREINANDO O MODELO

print("Gerando dados de amostra e treinando o modelo LightGBM...")
df_samples = generate_samples_melhorado()
X = df_samples[['NDVI', 'EVI', 'NDWI']]
y = df_samples['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = lgb.LGBMClassifier()
model.fit(X_train, y_train)

print("\nRelatório de Classificação do Modelo (em dados de teste):")
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=list(class_labels.values())))


In [ ]:
# CLASSIFICANDO AS IMAGENS
# Pode levar algum tempo...

classified_image_1, profile1 = processar_e_classificar_imagem(band_dir_data1, model)
classified_image_2, profile2 = processar_e_classificar_imagem(band_dir_data2, model)

# Verificação de Dimensões
if classified_image_1.shape != classified_image_2.shape:
    print("\nERRO: As imagens de entrada têm dimensões diferentes. A análise de mudança não pode continuar.")
    exit()


In [ ]:
# CALCULANDO A VARIAÇÃO DOS PIXELS ENTRE UMA IMAGEM E OUTRA
# Criamos um valor único para cada tipo de transição
# Ex: Vegetação (1) -> Solo Exposto (4) se torna o valor 14.
# Definindo as mudanças que queremos destacar e suas cores
# Formato: {codigo_mudanca: ('Descrição', 'cor')}
# Adicionamos uma categoria "Sem Mudança Significativa"
# Mapeando as mudanças de interesse no mapa

print("\nCalculando o mapa de mudanças...")
mapa_de_mudanca = np.zeros_like(classified_image_1)
mapa_de_mudanca = (classified_image_1 * 10) + classified_image_2

mudancas_de_interesse = {
    14: ('Vegetação -> Solo Exposto', 'red'),
    15: ('Vegetação -> Agricultura', 'orange'),
    41: ('Solo Exposto -> Vegetação', 'cyan'),
    51: ('Agricultura -> Vegetação', 'lime')
    
}

change_codes = [key for key in mudancas_de_interesse.keys()]
change_labels = [val[0] for val in mudancas_de_interesse.values()]
change_colors = [val[1] for val in mudancas_de_interesse.values()]


change_codes.append(0) # Usaremos 0 para tudo que não for de interesse
change_labels.append('Outras Mudanças / Sem Mudança')
change_colors.append('lightgray')


mapa_de_mudanca_visual = np.full(mapa_de_mudanca.shape, 0, dtype=int)
for code in mudancas_de_interesse.keys():
    mapa_de_mudanca_visual[mapa_de_mudanca == code] = code
    

In [ ]:
# VISUALIZAÇÃO DOS RESULTADOS DE CLASSIFICAÇÃO


print("Gerando e salvando visualizações...")


save_dir = r" Coloque o diretório que você deseja salvar as imagens aqui "
os.makedirs(save_dir, exist_ok=True)

fig, axs = plt.subplots(1, 2, figsize=(18, 9))

cmap_class = mcolors.ListedColormap(class_colors)
bounds_class = [0.5, 1.5, 2.5, 3.5, 4.5, 5.5]
norm_class = mcolors.BoundaryNorm(bounds_class, cmap_class.N)

axs[0].imshow(classified_image_1, cmap=cmap_class, norm=norm_class)
axs[0].set_title(f'Classificação - 22/08/2021 ({os.path.basename(os.path.dirname(os.path.dirname(band_dir_data1)))})', fontsize=16)
axs[0].axis('off')

axs[1].imshow(classified_image_2, cmap=cmap_class, norm=norm_class)
axs[1].set_title(f'Classificação - 05/03/2022 ({os.path.basename(os.path.dirname(os.path.dirname(band_dir_data2)))})', fontsize=16)
axs[1].axis('off')

cbar_ax = fig.add_axes([0.92, 0.25, 0.02, 0.5]) # Posição [esquerda, baixo, largura, altura]
cbar = fig.colorbar(plt.cm.ScalarMappable(cmap=cmap_class, norm=norm_class), cax=cbar_ax, ticks=[1, 2, 3, 4, 5])
cbar.ax.set_yticklabels(list(class_labels.values()))

plt.tight_layout(rect=[0, 0, 0.9, 1])

path_mapa = os.path.join(save_dir, "mapa_comparativo.png") # Nome do arquivo alterado
fig.savefig(path_mapa, dpi=300, bbox_inches='tight')
print(f"✔ Mapa comparativo salvo com sucesso em: {path_mapa}")

In [ ]:
# VISUALIZAÇÃO DO GRÁFICO DE MUDANÇAS

unique_changes, counts = np.unique(mapa_de_mudanca, return_counts=True)
change_stats = {change: count for change, count in zip(unique_changes, counts)}
mudancas_plot = {
    mudancas_de_interesse[code][0]: change_stats.get(code, 0)
    for code in mudancas_de_interesse
}
sorted_mudancas = sorted(mudancas_plot.items(), key=lambda item: item[1], reverse=True)
labels = [item[0] for item in sorted_mudancas]
values = [item[1] for item in sorted_mudancas]

fig_bar, ax_bar = plt.subplots(figsize=(10, 6))
bars = ax_bar.barh(labels, values, color='skyblue')
ax_bar.set_xlabel('Número de Pixels')
ax_bar.set_title('Contagem de Pixels por Tipo de Mudança Significativa')
ax_bar.invert_yaxis()
for bar in bars:
    ax_bar.text(bar.get_width() + 0.1, bar.get_y() + bar.get_height()/2, 
                f'{int(bar.get_width()):,}'.replace(',', '.'), va='center', ha='left')
plt.tight_layout()


path_grafico = os.path.join(save_dir, "grafico_contagem_mudancas.png")
fig_bar.savefig(path_grafico, dpi=300, bbox_inches='tight')
print(f"✔ Gráfico de contagem salvo com sucesso em: {path_grafico}")

plt.show()
